# Tweeter

In [1]:
import tweepy
import datetime

ModuleNotFoundError: No module named 'tweepy'

In [ ]:

# Authenticate to Twitter
CONSUMER_KEY = 'YOUR_API_KEY'
CONSUMER_SECRET = 'YOUR_API_SECRET_KEY'
ACCESS_TOKEN = 'YOUR_ACCESS_TOKEN'
ACCESS_TOKEN_SECRET = 'YOUR_ACCESS_TOKEN_SECRET'

auth = tweepy.OAuth1UserHandler(
    consumer_key=CONSUMER_KEY,
    consumer_secret=CONSUMER_SECRET,
    access_token=ACCESS_TOKEN,
    access_token_secret=ACCESS_TOKEN_SECRET
)

api = tweepy.API(auth)

# Define date range
end_date = datetime.datetime.now() 
start_date = end_date - datetime.timedelta(days=7)

# Search for tweets about Apple from the past week
tweets = tweepy.Cursor(api.search, q="Apple", since=start_date, until=end_date, lang="en").items()

# Print tweets
for tweet in tweets:
    print(tweet.text)


# Reddit

In [296]:
import praw
from datetime import datetime, timedelta
import pandas as pd
import csv
import time
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer

import re



#### set up token and test

In [4]:

# Authentication
reddit = praw.Reddit(
    client_id="bOUU3GVPswoqHAeWY-WB5g",
    client_secret="65G2ItcdbqJZU9pmGBC6303fYV-QJg",
    user_agent="script:sentimental_analysis:v1.0 (by /u/Legal_Advertising127)"
  #  username=''
)



In [21]:
subreddit = reddit.subreddit("python")

In [22]:
top_posts = subreddit.top(limit=10)
new_posts = subreddit.new(limit=10)

In [23]:
for post in top_posts:
    print('Title - ', post.title)
    print('ID - ', post.id)
    print('Author - ', post.author)
    print('URL - ', post.url)
    print('Score - ', post.score)
    print('Comment count - ', post.num_comments)
    print("Created -", post.created_utc)
    print("\n")
    
    

Title -  Lad wrote a Python script to download Alexa voice recordings, he didn't expect this email.
ID -  g53lxf
Author -  iEslam
URL -  https://i.redd.it/2s0dj8ob12u41.png
Score -  12341
Comment count -  133
Created - 1587424299.0


Title -  This post has:
ID -  hoolsm
Author -  Krukerfluk
URL -  https://www.reddit.com/r/Python/comments/hoolsm/this_post_has/
Score -  9237
Comment count -  437
Created - 1594386373.0


Title -  I redesign the Python logo to make it more modern
ID -  gftejm
Author -  jessjwilliamson
URL -  https://i.redd.it/rxezjyf4ojx41.png
Score -  7865
Comment count -  266
Created - 1588945149.0


Title -  Automate the boring stuff with python - tinder
ID -  7kpme8
Author -  backprop88
URL -  https://gfycat.com/PointlessSimplisticAmericanquarterhorse
Score -  6719
Comment count -  328
Created - 1513644476.0


Title -  Just finished programming and building my own smart mirror in python, wrote all of the code myself and implemented my own voice control and facial recog

In [11]:
for post in new_posts:
    print('Title - ', post.title)
    print('ID - ', post.id)
    print('Author - ', post.author)
    print('URL - ', post.url)
    print('Score - ', post.score)
    print('Comment count - ', post.num_comments)
    print("Created -", post.created_utc)
    print("\n")
    

Title -  Looking for contributors for my open-source AI security project
ID -  18d3rdq
Author -  kanxx030
URL -  https://www.reddit.com/r/Python/comments/18d3rdq/looking_for_contributors_for_my_opensource_ai/
Score -  1
Comment count -  0
Created - 1701978353.0


Title -  Pygolo 0.2.0: write Python extensions in Go!
ID -  18d3nt3
Author -  cavokz
URL -  https://www.reddit.com/r/Python/comments/18d3nt3/pygolo_020_write_python_extensions_in_go/
Score -  1
Comment count -  0
Created - 1701978092.0


Title -  CPython Dynamic Dispatch Internals: What Happens When You do a + b?
ID -  18d0axx
Author -  abhi9u
URL -  https://codeconfessions.substack.com/p/cpython-dynamic-dispatch-internals
Score -  2
Comment count -  0
Created - 1701969063.0


Title -  [Video] Explaining Floats Imprecision 🤖
ID -  18d05dh
Author -  JosephLovesPython
URL -  https://www.reddit.com/r/Python/comments/18d05dh/video_explaining_floats_imprecision/
Score -  0
Comment count -  0
Created - 1701968647.0


Title -  GitHub

#### collect data

In [342]:
# Basic rate limiting
def rate_limited_request(subreddits, companies, effective_preriod, post_limit=100, comment_limit=20):
    
    data = []
    for subreddit_name in subreddits:
        subreddit = reddit.subreddit(subreddit_name)
        try:
            for company in companies:
                count_post=0
                for post in subreddit.search(company, time_filter= effective_preriod):
                    #if post.created_utc >= timestamp:  # Ensure posts are only from the last week

                        # Convert the post's timestamp to the desired format (day month year)
                        post_time = datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d')
                        
                        # Add post title and body to the data list
                        data.append([post.title + "\n" + post.selftext, post_time, company])
                        count_post+=1
                        count_comment=0

                        try:
                            post.comments.replace_more(limit=0)  # Fetch top-level comments only
                            for comment in post.comments:
                                comment_time = datetime.utcfromtimestamp(comment.created_utc).strftime('%Y-%m-%d')
                                data.append([comment.body, comment_time, company])
                                count_comment+=1
                            if count_comment > 50:
                                break
                        except praw.exceptions.RedditAPIException as e:
                            print(f"Error fetching comments: {e}")
                            time.sleep(10)  # Sleep for 10 seconds before retrying
    
                        if count_post > post_limit:
                            break
        except praw.exceptions.RedditAPIException as e:
            print(f"Rate limit exceeded: {e}")
            time.sleep(10)  # Sleep for 10 seconds before retrying
    return data

def collect_data_reddit(companies, effective_preriod):
    # List of stock market-related subreddits
    subreddits = ["stocks", "investing", "wallstreetbets", "StockMarket", "options", "SecurityAnalysis", "Daytrading"]
    data = rate_limited_request(subreddits, companies, effective_preriod)
    return data


In [343]:

# Function to get wordnet POS tag
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

def clean_text(text):
    # Initialize the lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Tokenize the text
    tokens = word_tokenize(text)

    # Get English stop words
    stop_words = set(stopwords.words('english'))

    # Filter out the stop words, non-letter tokens, and lemmatize
    filtered_text = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens if word.isalpha() and word.lower() not in stop_words]
    #filtered_text = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tokens if word.lower() not in stop_words]


    # Rejoin filtered text
    filtered_sentence = ' '.join(filtered_text)

    return filtered_sentence

In [ ]:
def save_to_csv_reddit(data, company):

    df = pd.DataFrame(data, columns=["Text", "Date", "Company"])

    df['Text'] = df['Text'].apply(clean_text)

    df = df.groupby(['Company','Date']).agg({"Text": lambda x: ' '.join(x.astype(str))}).reset_index()

    save_file_name = 'data/reddit_'+company+'.csv'

    df.to_csv(save_file_name)

In [345]:
def clean_company_name(text):
    terms = ['Inc\.', 'Corporation', 'Company', 'plc', 'Limited', ',', 'and', '\.com', 'A/S', 'PLC', "'s"]
    pattern = r'(?:' + '|'.join(terms) + ')'
    # Replace the matched terms with an empty string
    cleaned_text = re.sub(pattern, '', text)
    return cleaned_text

In [346]:
def generate_top_100_company_name(path):
    companies_100 = pd.read_csv(path, index_col=0)
    companies_100 = companies_100[:100]
    companies_100['Company'] = companies_100['Company Name'].apply(clean_company_name)
    companies = list(companies_100['Company'].values)
    return companies

In [347]:
companies = generate_top_100_company_name('data/biggest-companies-stocks.csv')

In [ ]:
companies

In [348]:
data = collect_data_reddit(companies, 'month')

TooManyRequests: received 429 HTTP response

In [279]:
save_to_csv_reddit(data,"company")

In [ ]:
df_load = pd.read_csv('reddit_company.csv', index_col=0)